<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marcoteran/deeplearning/blob/master/notebooks/1.1_machinelearning_deeplearninglandscape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir en Colab" title="Abrir y ejecutar en Google Colaboratory"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marcoteran/deeplearning/blob/master/notebooks/1.1_machinelearning_deeplearninglandscape.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Abrir en Kaggle" title="Abrir y ejecutar en Kaggle"/></a>
  </td>
</table>

### Ejemplo de código
# Sesión 01: Proyecto integral de Machine Learning
## Deep Learning y series de tiempo

**Name:** Marco Teran
**E-mail:** marco.teran@usa.edu.co

[Website](http://marcoteran.github.io/),
[Github](https://github.com/marcoteran),
[LinkedIn](https://www.linkedin.com/in/marcoteran/).
___

In [ ]:
print("¡Bienvenidos al primer notebook!")

Este proyecto requiere Python 3.7 o superior:

In [ ]:
import sys

assert sys.version_info >= (3, 7)

It also requires Scikit-Learn ≥ 1.0.1:

In [ ]:
from packaging import version
import sklearn

print(version.parse(sklearn.__version__))

# Obtener los datos

Bienvenido a Machine Learning Housing Corp. Su tarea es predecir el valor medio de la vivienda en los distritos de California, dada una serie de características de estos distritos.

### Descargar los datos

In [ ]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()

## Eche un vistazo a la estructura de datos

Para empezar, se observan las cinco primeras filas de datos mediante el método head()

In [ ]:
housing.head()

Cada fila representa un distrito. Hay 10 atributos (no se muestran todos en la captura de pantalla): longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, median_house_value, y ocean_proximity.


El método info() es útil para obtener una descripción rápida de los datos, en particular el número total de filas, el tipo de cada atributo y el número de valores no nulos. no nulos:

In [ ]:
housing.info()

Hay 20.640 instancias en el conjunto de datos, lo que significa que es bastante pequeño para los estándares de aprendizaje automático, pero es perfecto para empezar. Observe que el atributo total_habitaciones sólo tiene 20.433 valores no nulos, lo que significa que 207 distritos carecen de esta característica. Tendrá que ocuparse de esto más adelante.

Todos los atributos son numéricos, excepto ocean_proximity.
El tipo de ocean_proximity es objeto, pero como se cargaron los datos desde un archivo CSV, se sabe que es un atributo de texto.
Al mirar las cinco filas superiores, se notó que los valores de la columna ocean_proximity eran repetitivos, lo que sugiere que se trata de un atributo categórico.
Se puede usar el método value_counts() para averiguar qué categorías existen y cuántos distritos pertenecen a cada categoría.

In [ ]:
housing["ocean_proximity"].value_counts()

Veamos los demás campos. El método describe() muestra un resumen de los atributos numéricos

In [ ]:
housing.describe()

La siguiente celda tampoco se muestra en el libro. Crea la carpeta `images/end_to_end_project` (si no existe ya), y define la función `save_fig()` que se utiliza a través de este cuaderno para guardar las figuras en alta resolución.

In [ ]:
# código extra - código para guardar las figuras como PNG de alta resolución para el libro

IMAGES_PATH = Path() / "images" / "end_to_end_project"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Una forma rápida de entender el tipo de datos con los que estás trabajando es graficar un histograma para cada atributo numérico. Un histograma muestra el número de instancias (en el eje vertical) que tienen un rango de valores dado (en el eje horizontal). Puedes graficar un solo atributo a la vez, o puedes llamar al método hist() en todo el conjunto de datos (como se muestra en el siguiente ejemplo de código), y generará un histograma para cada atributo numérico.

In [ ]:
import matplotlib.pyplot as plt

# código extra - las siguientes 5 líneas definen los tamaños de fuente por defecto
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

housing.hist(bins=50, figsize=(12, 8))
save_fig("attribute_histogram_plots")  # extra code
plt.show()

## Crear un conjunto de pruebas (Test Set)

Es recomendable reservar parte de los datos para formar un conjunto de prueba. Esto se debe a que si se mira el conjunto de prueba, se puede caer en la trampa del sobreajuste, y seleccionar un modelo que no funcionará tan bien como se esperaba.

Crear un conjunto de prueba es teóricamente simple; elige algunas instancias al azar, típicamente el 20% del conjunto de datos (o menos si tu conjunto de datos es muy grande), y resérvalos:

In [ ]:
import numpy as np

def shuffle_and_split_data(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = shuffle_and_split_data(housing, 0.2)
len(train_set)

In [ ]:
len(test_set)

Para asegurarnos de que los resultados de este cuaderno son los mismos cada vez que lo ejecutamos, necesitamos establecer la semilla aleatoria:

In [ ]:
np.random.seed(42)

Lamentablemente, esto no garantiza que este cuaderno produzca exactamente los mismos resultados que el libro, ya que hay otras posibles fuentes de variación. La más importante es el hecho de que los algoritmos se modifican con el tiempo cuando las bibliotecas evolucionan. Así que, por favor, tolera algunas pequeñas diferencias: con suerte, la mayoría de los resultados deberían ser los mismos, o al menos aproximados.

Nota: otra fuente de aleatoriedad es el orden de los conjuntos de Python: se basa en la función `hash()` de Python, que es "salada" aleatoriamente cuando Python arranca (esto empezó en Python 3.3, para prevenir algunos ataques de denegación de servicio). Para eliminar esta aleatoriedad, la solución es establecer la variable de entorno `PYTHONHASHSEED` a `"0"` _antes_ de que Python se inicie. No pasará nada si lo haces después. Afortunadamente, si está ejecutando este cuaderno en Colab, la variable ya está configurada para usted.

In [ ]:
from zlib import crc32

def is_id_in_test_set(identifier, test_ratio):
    return crc32(np.int64(identifier)) < test_ratio * 2**32

def split_data_with_id_hash(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: is_id_in_test_set(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

Lamentablemente, el conjunto de datos sobre vivienda no dispone de una columna de identificadores. La solución más sencilla de solución más sencilla es utilizar el índice de la fila como identificador:

In [ ]:
housing_with_id = housing.reset_index()  # adds an `index` column
train_set, test_set = split_data_with_id_hash(housing_with_id, 0.2, "index")

Si se utiliza el índice de filas como identificador único, hay que asegurarse de que que los nuevos datos se añadan al final del conjunto de datos y que nunca se fila. Si esto no es posible, puede intentar utilizar las características más estables para construir un identificador único. Por ejemplo, está garantizado que la latitud y la longitud de un distrito se mantendrán estables durante varios millones de años:

In [ ]:
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_data_with_id_hash(housing_with_id, 0.2, "id")

Scikit-Learn proporciona algunas funciones para dividir conjuntos de datos en múltiples subconjuntos de varias formas. La función más simple es train_test_split(), que hace prácticamente lo mismo que la función shuffle_and_split_data() que definimos anteriormente, pero con un par de características adicionales. Primero, hay un parámetro random_state que te permite establecer la semilla del generador aleatorio. En segundo lugar, puedes pasarle varios conjuntos de datos con un número idéntico de filas y los dividirá en los mismos índices (esto es muy útil, por ejemplo, si tienes un DataFrame separado para etiquetas).

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
test_set["total_bedrooms"].isnull().sum()

Para hallar la probabilidad de que una muestra aleatoria de 1.000 personas contenga menos del 48,5% de mujeres o más del 53,5% de mujeres cuando la proporción de mujeres de la población es del 51,1%, utilizamos la [distribución binomial](https://en.wikipedia.org/wiki/Binomial_distribution). El método `cdf()` de la distribución binomial nos da la probabilidad de que el número de mujeres sea igual o menor que el valor dado.

In [ ]:
# código adicional - muestra cómo calcular la probabilidad del 10,7% de obtener una muestra errónea

from scipy.stats import binom

sample_size = 1000
ratio_female = 0.511
proba_too_small = binom(sample_size, ratio_female).cdf(485 - 1)
proba_too_large = 1 - binom(sample_size, ratio_female).cdf(535)
print(proba_too_small + proba_too_large)

Si prefieres las simulaciones a las matemáticas, aquí tienes cómo obtener aproximadamente el mismo resultado:

In [ ]:
# código extra - muestra otra forma de estimar la probabilidad de mala muestra

np.random.seed(42)

samples = (np.random.rand(100_000, sample_size) < ratio_female).sum(axis=1)
((samples < 485) | (samples > 535)).mean()

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
housing["income_cat"].value_counts().sort_index().plot.bar(rot=0, grid=True)
plt.xlabel("Categoría de ingresos")
plt.ylabel("Número de distritos")
save_fig("housing_income_cat_bar_plot") # código adicional
plt.show()

Para ser precisos, el método split() proporciona los índices de entrenamiento y prueba, no los datos en sí. Tener múltiples divisiones puede ser útil si desea estimar mejor el rendimiento de su modelo, como verá cuando hablemos de la validación cruzada más adelante en este capítulo. Por ejemplo, el siguiente código genera 10 divisiones estratificadas diferentes del mismo conjunto de datos:

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
strat_splits = []
for train_index, test_index in splitter.split(housing, housing["income_cat"]):
    strat_train_set_n = housing.iloc[train_index]
    strat_test_set_n = housing.iloc[test_index]
    strat_splits.append([strat_train_set_n, strat_test_set_n])

Por ahora, sólo puedes utilizar la primera división:

In [ ]:
strat_train_set, strat_test_set = strat_splits[0]

Puesto que el muestreo estratificado es bastante común, hay una forma más corta de obtener una utilizando la función train_test_split() con el argumento stratify. Es mucho más corto obtener una única división estratificada:

In [ ]:
strat_train_set, strat_test_set = train_test_split(
    housing, test_size=0.2, stratify=housing["income_cat"], random_state=42)

Veamos si ha funcionado como se esperaba. Para empezar, observe las proporciones de en el conjunto de pruebas:

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

Con un código similar, se puede medir la proporción de categorías de ingresos en el conjunto de datos completo. La tabla compara las proporciones de categorías de ingresos en el conjunto de datos completo, en el conjunto de prueba generado con muestreo estratificado y en un conjunto de prueba generado utilizando un muestreo completamente aleatorio. Como se puede ver, el conjunto de prueba generado utilizando muestreo estratificado tiene proporciones de categorías de ingresos casi idénticas a las del conjunto de datos completo, mientras que el conjunto de prueba generado utilizando muestreo completamente aleatorio está sesgado.

In [ ]:
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall %": income_cat_proportions(housing),
    "Stratified %": income_cat_proportions(strat_test_set),
    "Random %": income_cat_proportions(test_set),
}).sort_index()
compare_props.index.name = "Income Category"
compare_props["Strat. Error %"] = (compare_props["Stratified %"] /
                                   compare_props["Overall %"] - 1)
compare_props["Rand. Error %"] = (compare_props["Random %"] /
                                  compare_props["Overall %"] - 1)
(compare_props * 100).round(2)

No volverás a utilizar la columna income_cat, así que será mejor que la elimines, volviendo los datos a su estado original: for set_ in (strat_train_set, strat_test_set)

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

Dedicamos bastante tiempo a la generación de conjuntos de pruebas por una buena razón: se trata de una parte a menudo descuidada pero fundamental de un proyecto de aprendizaje automático. parte crítica de un proyecto de aprendizaje automático. Además, muchas de estas ideas serán útiles más adelante cuando hablemos de la validación cruzada.
Ahora es el momento de pasar a la siguiente etapa: explorar los datos.

# Descubrir y visualizar los datos para obtener información (insights)

Es necesario una exploración más detallada de los datos y sugiere tomar medidas específicas antes de explorar los datos en profundidad. Primero, se recomienda que separe el conjunto de prueba y solo explore el conjunto de entrenamiento. Además, si el conjunto de entrenamiento es grande, puede ser útil muestrear un conjunto de exploración para facilitar y acelerar la exploración. En este caso, el conjunto de entrenamiento es pequeño, por lo que se puede trabajar directamente con el conjunto completo. Dado que se experimentará con varias transformaciones en el conjunto de entrenamiento completo, se sugiere hacer una copia del original para poder volver a él más tarde.

In [ ]:
housing = strat_train_set.copy()

## Visualización de datos geográficos

Dado que el conjunto de datos incluye información geográfica (latitud y longitud), conviene crear un diagrama de dispersión de todos los distritos para visualizar los datos.

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", grid=True)
save_fig("bad_visualization_plot")  # extra code
plt.show()

Esto se parece a California, pero aparte de eso es difícil ver un patrón particular. patrón particular. Establecer la opción alfa a 0,2 hace que sea mucho más fácil de visualizar los lugares donde hay una alta densidad de puntos de datos

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", grid=True, alpha=0.2)
save_fig("better_visualization_plot")  # extra code
plt.show()

A continuación, observa los precios de la vivienda. El radio de cada círculo representa la población del distrito (opción s), y el color representa el precio (opción c). Aquí se utiliza un mapa de colores predefinido (opción cmap) llamado jet, que va del azul (valores bajos) al rojo (precios altos):

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", grid=True,
             s=housing["population"] / 100, label="population",
             c="median_house_value", cmap="jet", colorbar=True,
             legend=True, sharex=False, figsize=(10, 7))
save_fig("housing_prices_scatterplot")  # extra code
plt.show()

El argumento `sharex=False` corrige un error de visualización: sin él, los valores del eje x y la etiqueta no se muestran (véase: https://github.com/pandas-dev/pandas/issues/10611).

La imagen muestra que los precios de las viviendas están relacionados con la ubicación y la densidad de población. Un algoritmo de agrupación podría ayudar a detectar el conglomerado principal y agregar nuevas características que midan la proximidad a los centros de los conglomerados. El atributo de proximidad al océano también puede ser útil, pero en el norte de California los precios de la vivienda en los distritos costeros no son demasiado altos.

## Buscando Correlaciones

Como el conjunto de datos no es demasiado grande, puede calcular fácilmente el coeficiente de correlación estándar (también llamado r de Pearson) entre cada par de atributos utilizando el método corr():

In [ ]:
corr_matrix = housing.corr()

Ahora puede ver en qué medida cada atributo está correlacionado con la mediana de la vivienda:

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

El resultado indica que el coeficiente de correlación oscila entre -1 y 1, donde valores cercanos a 1 indican una fuerte correlación positiva y valores cercanos a -1 indican una fuerte correlación negativa. En el caso de la mediana de los ingresos y el valor medio de la vivienda, hay una correlación positiva, mientras que hay una pequeña correlación negativa entre la latitud y el valor medio de la vivienda. Si el coeficiente se acerca a 0, significa que no hay correlación lineal.

Para comprobar la correlación entre los atributos, se puede utilizar la función Pandas scatter_matrix(), que compararía cada atributo numérico con cada otro atributo numérico. Dado que hay 11 atributos numéricos, se deben centrar en algunos atributos que parezcan estar más correlacionados con el valor medio de la vivienda.

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
save_fig("scatter_matrix_plot")  # extra code
plt.show()

La diagonal principal estaría llena de líneas rectas si Pandas trazara cada contra sí misma, lo que no sería muy útil. Así que en su lugar, Pandas muestra un histograma de cada atributo (hay otras opciones disponibles; consulte la documentación de Pandas para más detalles).

Observando los diagramas de correlación, parece que el atributo más prometedor para predecir el valor medio de la vivienda es la renta media. para predecir el valor medio de la casa es la mediana de los ingresos, por lo que se ampliar su diagrama de dispersión

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1, grid=True)
save_fig("income_vs_house_value_scatterplot")  # extra code
plt.show()

## Experimenting with Attribute Combinations

Se ha explorado los datos para identificar peculiaridades y correlaciones entre atributos antes de alimentar los datos a un algoritmo de aprendizaje automático. También se ha observado que algunos atributos tienen una distribución sesgada a la derecha y se sugiere transformarlos para preparar los datos para los algoritmos de aprendizaje automático. Además, se ha recomendado probar varias combinaciones de atributos antes de preparar los datos y se sugieren nuevos atributos como el número de habitaciones por hogar y la población por hogar. Se deja planteado cómo se crearán estos nuevos atributos.

In [ ]:
housing["rooms_per_house"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_ratio"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["people_per_house"] = housing["population"] / housing["households"]

Nuevamente la matriz de correlación:

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

# Preparar los datos para los algoritmos de aprendizaje automático

* Volver a un conjunto de entraniemnto limpio (copiando otra vez `strat_train_set`)
* Separar los predictores y las etiquetas (no es necesario aplicar las mismas transformaciones)

Volvamos al conjunto de entrenamiento original y separemos el objetivo (ten en cuenta que `strat_train_set.drop()` crea una copia de `strat_train_set` sin la columna, en realidad no modifica `strat_train_set` en sí, a menos que pases `inplace=True`):

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

## Limpiar los datos (Data Cleaning)

La mayoría de los algoritmos de ***machine learning*** no pueden funcionar si faltan caracteristicas. Crearemos funciones que se ocupen de ello.
* Notamos que a `total_bedrooms` le faltan valores
Tenemos tres opciones:
1. Deshacernos de los distritos correspondientes
2. Deshacernos de todo el atributo
3. Establecer algún valor para esos valores (cero, la media, la mediana, etc.

Se enumeran 3 opciones para manejar los valores NaN:

```python
housing.dropna(subset=["total_bedrooms"], inplace=True)    # option 1

housing.drop("total_bedrooms", axis=1)       # option 2

median = housing["total_bedrooms"].median()  # option 3
housing["total_bedrooms"].fillna(median, inplace=True)
```
Para cada opción, crearemos una copia de `housing` y trabajaremos sobre esa copia para evitar romper `housing`. También mostraremos la salida de cada opción, pero filtrando las filas que originalmente contenían un valor NaN.

In [ ]:
null_rows_idx = housing.isnull().any(axis=1)
housing.loc[null_rows_idx].head()

In [ ]:
housing_option1 = housing.copy()

housing_option1.dropna(subset=["total_bedrooms"], inplace=True)  # option 1

housing_option1.loc[null_rows_idx].head()

In [ ]:
housing_option2 = housing.copy()

housing_option2.drop("total_bedrooms", axis=1, inplace=True)  # option 2

housing_option2.loc[null_rows_idx].head()

In [ ]:
housing_option3 = housing.copy()

median = housing["total_bedrooms"].median()
housing_option3["total_bedrooms"].fillna(median, inplace=True)  # option 3

housing_option3.loc[null_rows_idx].head()

Scikit-Learn ofrece una clase útil para ocuparse de los valores que faltan `SimpleImputer`:
* Se debe crear una instancia `SimpleImputer`

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

Separar los atributos numéricos de los no númericos para utilizar la estrategia `"median"`. No se puede calcular sobre atributos de texto como `ocean_proximity`:

In [ ]:
imputer.strategy

In [ ]:
housing_num = housing.select_dtypes(include=[np.number])

Ahora se pude ajustar la instancia `imputer` a los datos de entrenamiento utilizando el método `fit()`:

In [ ]:
imputer.fit(housing_num)

La instancia `imputer` calcula la media de cada atributo y ha almacenado el resultado en su variable de instancia `statistics_`

In [ ]:
imputer.statistics_

Solo le faltaban valores al atributo `total_bedrooms`, pero no podemos estar seguros de que no faltarán valores en datos nuevos después de que se lance el sistema. Es más seguro aplica `imputer` a todos los atributos númericos.
Compruebe que es lo mismo que calcular manualmente la mediana de cada atributo:

In [ ]:
housing_num.median().values

Transformar el conjunto de entrenamiento utilizando la `imputer` entrenada para todo el datos:

In [ ]:
X = imputer.transform(housing_num)

El resultado es una matriz NumPy sencila que contiene las características transformadas.
Si se quiere volver a ponerla en DataFrame de pandas es necesario:

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing_num.index)

Se puede comprombar que la transformación se realizó con éxito:

In [ ]:
housing_tr.loc[null_rows_idx].head()

## Eliminando los ***outliers***

Isolation Forest es un algoritmo de detección de anomalías basado en árboles de decisión aleatorios.
- Utiliza una estrategia de particionamiento aleatorio para construir árboles de decisión de forma rápida y eficiente.
- Para detectar una anomalía, mide la profundidad del árbol necesario para aislarla de las observaciones normales.
- Es un algoritmo rápido y escalable que puede manejar valores faltantes y datos categóricos en conjuntos de datos de alta dimensionalidad.

In [ ]:
from sklearn.ensemble import IsolationForest

isolation_forest = IsolationForest(random_state=42)
outlier_pred = isolation_forest.fit_predict(X)

In [ ]:
outlier_pred

Si desea eliminar los valores atípicos, ejecute el código siguiente:

In [ ]:
#housing = housing.iloc[outlier_pred == 1]
#housing_labels = housing_labels.iloc[outlier_pred == 1]

## Tratamiento de atributos de texto y categóricos

Hasta ahora se han manejado atributos numéricos. Existen atributos de texto.

Ahora vamos a preprocesar la característica categórica de entrada, `ocean_proximity`.
Revisemos las primeras 8 instancias:

In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(8)

- No es un texto arbitrario, hay un conjunto limitado de valores posibles de texto
- Cada uno de los cuales representa una categoría, por tanto es un atributo categórico.

La mayoría de los algoritmos de ***machine learning*** prefiere trabajar con números. Así que se procede a convertir las catergorías de texto en números. Se utiliza la clase `OrdinalEncoder`:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

In [ ]:
housing_cat_encoded[:8]

Lista de categorías procesadas:

In [ ]:
ordinal_encoder.categories_

* Un problema de esta representación es que los algoritmos de ***machine laerning*** asumen que dos valores ceranos son más similares que dos valores distantes
* Para solucionar este inconveniente, la mejor opción es crear un atributo binario ***one vs all***

La codificación **one-hot** es un proceso mediante el cual se convierten variables categóricas en vectores binarios para que puedan ser utilizados como entradas en algoritmos de aprendizaje automático.
* La codificación **one-hot** asigna un valor binario de 1 a la categoría presente y 0 para las demás, creando un vector que representa la presencia o ausencia de cada categoría.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

In [ ]:
housing_cat_1hot

La salida de `OneHotEncoder` es una matriz dispersa de SciPy, en lugar de una matriz de NumPy.
Para convertirla en una matriz densa, se utiliza el método `toarray()`:

In [ ]:
housing_cat_1hot.toarray()

In [ ]:
cat_encoder.categories_

Alternativamente, puede establecer `sparse=False` al crear el `OneHotEncoder`:

In [ ]:
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
cat_encoder.categories_

## Escalado de características

Una dde las transformaciones más importantes que hay que aplciar en los datos, salvo algunas excepeciones, el rendimiento no es bueno si no se hace un escalado previo.

Hay dos formas de obtener la misma escala:
1. Escalado min/max, llamado también normalización: los valores se trasladan y escalan para quedar entre 0 y 1. Sustrae el valor mínimo y divide entre el máximo menos el mínimo. Transformador: `MinMaxScaler`
2. Normalización sustrae el valor medio (media cero) y luego se divide por la desviación estándar (resultado final con varianza unitaria). No hay rango de límites. Transformador: `StandardScaler`

In [ ]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
housing_num_min_max_scaled = min_max_scaler.fit_transform(housing_num)

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
housing_num_std_scaled = std_scaler.fit_transform(housing_num)

## Pipelines de transformación

Hay muchos pasaso de transformaciónd edatos que hay que ejecutrar en el orden correcto.
Scikit-Learn ofrece la clase `Pipeline` para ayudar con estas secuencias de transformaciones.

Ahora vamos a construir un pipeline sencillo para preprocesar los atributos numéricos:

In [ ]:
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler()),
])

`make_pipeline` es una función de Scikit-learn que permite crear un objeto de tubería (pipeline) para encadenar varios pasos de procesamiento de datos y un estimador final en un solo objeto:

In [ ]:
from sklearn.pipeline import make_pipeline

num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

`set_config`  permite establecer opciones de configuración globales para todos los estimadores y transformadores de la biblioteca:

In [ ]:
from sklearn import set_config

set_config(display='diagram')

num_pipeline

Finalmente `fit_transform` es un método que se utiliza para ajustar y transformar los datos utilizando un objeto de tubería (pipeline) que encadena varios pasos de procesamiento de datos y un estimador final. Este método ajusta cada transformador en el pipeline a los datos de entrenamiento y transforma los datos de entrenamiento utilizando todos los transformadores en el orden especificado en el pipeline, y finalmente ajusta el estimador final a los datos transformados.

In [ ]:
housing_num_prepared = num_pipeline.fit_transform(housing_num)
housing_num_prepared[:2].round(2)

¡Todo bien! ¡Es todo por hoy! 😀